<a href="https://www.kaggle.com/code/pranayrishith16/nyc-trip-duration?scriptVersionId=134720566" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians, sin, cos, sqrt, atan2
from geopy.distance import geodesic
from sklearn.cluster import KMeans
from haversine import haversine
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor


import folium
import zipfile


np.random.seed(42)

In [ ]:
with zipfile.ZipFile('/kaggle/input/nyc-taxi-trip-duration/train.zip','r') as zip_ref:
    zip_ref.extractall()

In [ ]:
%%time
data = pd.read_csv('/kaggle/working/train.csv')
data.set_index('id',inplace=True)
data.head(20)

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
pd.set_option("display.float_format", "{:.2f}".format)
data.describe()

In [ ]:
data.info()

In [ ]:
data = data.loc[data['trip_duration']<20000]

There are outliers here. This might not comply with our obervation. No one takes 2 million seconds in cab travelling.

In [ ]:
sns.displot(x=data['trip_duration'])

In [ ]:
sns.countplot(data=data,x='vendor_id')

We have 2 vendors.

Pickup and dropoff are actually objects but i need them to be datetime type. lets convert it.

In [ ]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'],format='%Y-%m-%d %H:%M:%S')

Lets plot the latitudes and longitudes on the map.

In [ ]:
# f, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(15,10))

# sns.scatterplot(data=data,x='pickup_longitude', y='pickup_latitude',color='#F2F7F2', ax=ax1,s=.07, alpha=.6)
# ax1.set_title("Pickups")
# ax1.set_facecolor('#040926')

# sns.scatterplot(data=data,x='dropoff_longitude', y='dropoff_latitude',color='#F2F7F2', ax=ax2,s=.07, alpha=.6)
# ax2.set_title("Dropoffs")
# ax2.set_facecolor('#040926')
# plt.show()

Nice looking at the maps. You might seem skeptical that the above figure arent maps, but trust me this is a map. The latitudes and longitudes are lot scattered so you cant see the map right. Try weird viewing angles to see the outliers. I dont think that comes into NYC. So lets do some filtering and try the map visualizing again.

Based on research, I am implying some boundries.

**Latitude ranges from 40.915256 to 40.477044
Longitudes ranges from -74.005735 to -73.700272**

In [ ]:
data = data.loc[(data['pickup_latitude'] >= 40.637044) & (data['pickup_latitude'] <= 40.855256)]
data = data.loc[(data['pickup_longitude'] >= -74.035735) & (data['pickup_longitude'] <= -73.770272)]
data = data.loc[(data['dropoff_latitude'] >= 40.637044) & (data['dropoff_latitude'] <= 40.855256)]
data = data.loc[(data['dropoff_longitude'] >= -74.035735) & (data['dropoff_longitude'] <= -73.770272)]
data.shape

In [ ]:
# f, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(15,10))

# sns.scatterplot(data=data,x='pickup_longitude', y='pickup_latitude',color='#F2F7F2', ax=ax1,s=.07, alpha=.6)
# ax1.set_title("Pickups")
# ax1.set_facecolor('#040926')

# sns.scatterplot(data=data,x='dropoff_longitude', y='dropoff_latitude',color='#F2F7F2', ax=ax2,s=.07, alpha=.6)
# ax2.set_title("Dropoffs")
# ax2.set_facecolor('#040926')
# plt.show()

If you obeserve the map, most of the pickups and dropoffs are downtown of manhattan. and you see something on the right bottom corner, its JFK airport. The dropoffs are scattered in manhattan and near manhattan. So if the dropoffs is manhattan or via manhattan then at peak time the duration will be a lot.

In [ ]:
data['day_of_week'] = data['pickup_datetime'].dt.dayofweek
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data['day_of_week'] = data['day_of_week'].map(lambda x: day_names[x])
data['day_of_week']

In [ ]:
sns.boxplot(data=data,x='day_of_week',y='trip_duration')
plt.ticklabel_format(style='plain', axis='y')
plt.show()

In [ ]:
data['trip_in_minutes'] = data['trip_duration']/60
sns.boxplot(data=data,x='day_of_week',y='trip_in_minutes')
plt.ticklabel_format(style='plain', axis='y')
plt.show()

There are outliers here. This might not comply with our obervation. No one takes 30000 minutes in cab travelling.

Lets see rushhour trips. Based on research

**Morning hours: 6:00 AM to 11:00 AM\
Afternoon hours: 11:00 AM to 4:00 PM\
Evening hours: 4:00 PM to 8:00 PM \
Night hours: 8:00 PM to 2:00 AM**

In [ ]:
data['pickup_hour'] = data['pickup_datetime'].dt.hour
data['pickup_hour'].head()

In [ ]:
sns.countplot(data=data,x='pickup_hour')

There is a surge of pickups from night 6:00 PM onwards. and peak is around 6:00 PM to 8:00PM. and the peak trend follows until midnight. Early morning starts at 6:00 AM and increases. Early morning, its time to go to work and evening is time to come home or go out.

Lets check out monthly trips.

In [ ]:
data['pickup_month'] = data['pickup_datetime'].dt.month

sns.countplot(data=data,x='pickup_month')

Its strange to see but there is only 6 months of data and August is starting of tourist season but this data doesnt have next 6 months.

In [ ]:
data['date'] = data['pickup_datetime'].dt.day
plt.figure(figsize=(14,8))
sns.countplot(data=data,x='date',hue='vendor_id')

There's not a lot to observe here.

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=data,x='day_of_week',hue='vendor_id')

Considerbly, weekends have more trips.

In [ ]:
average_trip = data.groupby('day_of_week')['trip_duration'].mean()
plt.bar(average_trip.index,average_trip.values)

In [ ]:
data.columns

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.corr(),annot=True)
plt.show()

We have latitudes and longitudes which we cannot use directly in modeling. So I am going to experiment those into clusters.

In [ ]:
pickup_coordinates = data[['pickup_latitude','pickup_longitude']]
dropoff_coordinates = data[['dropoff_latitude','dropoff_longitude']]

n_clusters = 5

pickup_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
data['pickup_cluster_label'] = pickup_kmeans.fit_predict(pickup_coordinates)

dropoff_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
data['dropoff_cluster_label'] = dropoff_kmeans.fit_predict(dropoff_coordinates)

In [ ]:
# palatte = ["#22181C","#835D6A","#E3170A","#2E86AB","#38A700"]
# palette = sns.color_palette(palatte, as_cmap=True)

# sns.scatterplot(data=data,x='pickup_longitude',y='pickup_latitude',hue='pickup_cluster_label',s=.07, alpha=.6,palette=palatte)
# plt.show()

In [ ]:
# sns.scatterplot(data=data,x='dropoff_longitude',y='dropoff_latitude',hue='dropoff_cluster_label',s=.07, alpha=.6,palette=palatte)
# plt.show()

In [ ]:
average_pickup_time = data.groupby('pickup_cluster_label')['trip_in_minutes'].mean()
plt.bar(average_pickup_time.index,average_pickup_time.values)

Cluster 2 has more trip time as the pickup is JFK airport.

In [ ]:
average_dropoff_time = data.groupby('dropoff_cluster_label')['trip_in_minutes'].mean()
plt.bar(average_dropoff_time.index,average_dropoff_time.values)

Cluster 2 has more trip time as the dropoff is JFK airport.

In [ ]:
sns.countplot(data=data,x='pickup_cluster_label')

Upper downtown has more pickups

In [ ]:
sns.countplot(data=data,x='dropoff_cluster_label')

Again upper downtown has more dropoffs.

In [ ]:
data['haversine_distance'] = data.apply(lambda x: haversine((x['pickup_latitude'],x['pickup_longitude']),(x['dropoff_latitude'],x['dropoff_longitude'])),axis=1)
data['haversine_distance'].head(10)

In [ ]:
averageDis = data.groupby('day_of_week')['haversine_distance'].mean()
plt.bar(averageDis.index,averageDis.values)

In [ ]:
sns.pointplot(x='day_of_week',y='haversine_distance',hue='vendor_id',data=data)

In [ ]:
data.describe()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data.corr(),annot=True,fmt='.2f')

In [ ]:
data.drop(['trip_in_minutes','pickup_datetime','dropoff_datetime'],axis=1,inplace=True)
data.columns

Thats it for understanding the data, lets model it.

## Data Modeling

In [ ]:
data = pd.get_dummies(data,columns=['vendor_id','store_and_fwd_flag','day_of_week'])
data.columns

In [ ]:
y = data['trip_duration']
X = data.drop(['trip_duration'],axis=1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.3)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
model = RandomForestRegressor(max_depth=8)

model.fit(X_train,y_train)

In [ ]:
importances = model.feature_importances_
feature_names = X.columns
indices = np.argsort(importances)[::-1]
sorted_feature_names = [feature_names[i] for i in indices]
sorted_importances = importances[indices]

plt.figure(figsize=(10, 6))
plt.bar(range(len(sorted_importances)), sorted_importances, align='center')
plt.xticks(range(len(sorted_importances)), sorted_feature_names, rotation='vertical')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.tight_layout()

In [ ]:
ypred = model.predict(X_test)

print("MSE: ",mean_squared_error(y_test,ypred))

In [ ]:
adamodel = AdaBoostRegressor()
adamodel.fit(X_train,y_train)

ypred = adamodel.predict(X_test)

print("MSE: ",mean_squared_error(y_test,ypred))

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

xgb_pars = {'min_child_weight': 10, 'eta': 0.04, 'colsample_bytree': 0.8, 'max_depth': 15,
            'subsample': 0.75, 'lambda': 2, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'gamma' : 0,
            'eval_metric': 'rmse', 'objective': 'reg:squarederror'}    

model = xgb.train(xgb_pars, dtrain, 500, watchlist, early_stopping_rounds=250,
                  maximize=False, verbose_eval=15)